In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [52]:
# Parameters
batch_size = 32
image_size = 64
learning_rate = 0.001
num_epochs = 20

In [34]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [37]:
from google.colab import files
uploaded = files.upload()


Saving homer_bart.zip to homer_bart (2).zip


In [40]:
import zipfile
import os

# Getting the name of the uploaded file
filename = list(uploaded.keys())[0]

# Extracting the zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('homer_bart')

# Listing the files to verify extraction
os.listdir('homer_bart')


['Bart', 'Homer']

In [41]:
# Path to dataset
dataset_path = 'homer_bart'  # Path after extraction

In [42]:
# Loading dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)


In [43]:
# Splitting dataset into training and testing
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [44]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [45]:
# Defining the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(image_size * image_size * 3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [46]:
model = SimpleNN()


In [47]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [53]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        labels = labels.view(-1, 1).float()  # Reshaping labels for BCELoss
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/20], Loss: 0.0147
Epoch [2/20], Loss: 0.0123
Epoch [3/20], Loss: 0.0072
Epoch [4/20], Loss: 0.0078
Epoch [5/20], Loss: 0.0064
Epoch [6/20], Loss: 0.0036
Epoch [7/20], Loss: 0.0020
Epoch [8/20], Loss: 0.0025
Epoch [9/20], Loss: 0.0018
Epoch [10/20], Loss: 0.0017
Epoch [11/20], Loss: 0.0006
Epoch [12/20], Loss: 0.0008
Epoch [13/20], Loss: 0.0007
Epoch [14/20], Loss: 0.0013
Epoch [15/20], Loss: 0.0009
Epoch [16/20], Loss: 0.0004
Epoch [17/20], Loss: 0.0002
Epoch [18/20], Loss: 0.0006
Epoch [19/20], Loss: 0.0005
Epoch [20/20], Loss: 0.0005


In [54]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        labels = labels.view(-1, 1).float()  # Reshaping labels for BCELoss
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 85.19%
